In [ ]:
# non_linear_model
modeldeepchem_gslf = create_linear_classifier(gs_lf_tasks, 1,dataset_gslf)
metric_gslf = dc.metrics.Metric(dc.metrics.roc_auc_score)

modeldeepchem_gslf,train_scores_gslf,test_scores_gslf,losses_gslf= run_linear_classifier(modeldeepchem_gslf,embedding_train_dataset_gslf,embedding_test_dataset_gslf,metric_gslf,'roc_auc_score',nb_epoch = 100)



In [ ]:
def prepare_ravia_similarity_mols2(df_ravia_similarity_mean,lm,tokenizer,modeldeepchem_gslf):
    df_ravia_mean_mols1 = df_ravia_similarity_mean[['Stimulus 1-IsomericSMILES','Stimulus 1-nonStereoSMILES','CID Stimulus 1']].drop_duplicates().reset_index(drop=True)
    df_ravia_mean_mols2 = df_ravia_similarity_mean[['Stimulus 2-IsomericSMILES','Stimulus 2-nonStereoSMILES','CID Stimulus 2']].drop_duplicates().reset_index(drop=True).rename(columns={'Stimulus 2-nonStereoSMILES': 'Stimulus 1-nonStereoSMILES','Stimulus 2-IsomericSMILES':'Stimulus 1-IsomericSMILES', 'CID Stimulus 2': 'CID Stimulus 1' })
    df_ravia_mols= pd.concat([df_ravia_mean_mols1, df_ravia_mean_mols2], ignore_index=True, axis=0).reset_index(drop=True)
    df_ravia_mols=df_ravia_mols.drop_duplicates().reset_index(drop=True)
    df_ravia_mols = df_ravia_mols.rename(columns={'Stimulus 1-IsomericSMILES': 'IsomericSMILES','Stimulus 1-nonStereoSMILES':'nonStereoSMILES', 'CID Stimulus 1': 'CID' })
    mol_type="nonStereoSMILES"


    res=prepare_mols_helper(lm,tokenizer,df_ravia_mols,modeldeepchem=modeldeepchem_gslf)
    if modeldeepchem_gslf is not None:
        df_ravia_mols_embeddings_original,df_ravia_mols_layers_original,df_ravia_mols_embeddings,df_ravia_mols_embeddings_zscored,df_ravia_mols_layers,df_ravia_mols_layers_zscored,df_ravia_mols_embeddings_linear,df_ravia_mols_embeddings_linear_zscored=res
        return df_ravia_mols,df_ravia_mols_embeddings_original,df_ravia_mols_layers_original,df_ravia_mols_embeddings,df_ravia_mols_embeddings_zscored,df_ravia_mols_layers,df_ravia_mols_layers_zscored,df_ravia_mols_embeddings_linear,df_ravia_mols_embeddings_linear_zscored
    else:
        df_ravia_mols_embeddings_original,df_ravia_mols_layers_original,df_ravia_mols_embeddings,df_ravia_mols_embeddings_zscored,df_ravia_mols_layers,df_ravia_mols_layers_zscored=res
        return df_ravia_mols,df_ravia_mols_embeddings_original,df_ravia_mols_layers_original,df_ravia_mols_embeddings,df_ravia_mols_embeddings_zscored,df_ravia_mols_layers,df_ravia_mols_layers_zscored
    # return df_ravia_mols,df_ravia_mols_embeddings_original,df_ravia_mols_layers_original,df_ravia_mols_embeddings,df_ravia_mols_embeddings_zscored,df_ravia_mols_layers,df_ravia_mols_layers_zscored,df_ravia_mols_embeddings_linear,df_ravia_mols_embeddings_linear_zscored


In [ ]:
# df_ravia_similarity_mols,df_ravia_similarity_mols_embeddings_original,df_ravia_similarity_mols_layers_original\
# ,df_ravia_similarity_mols_embeddings,df_ravia_similarity_mols_embeddings_zscored,\
# df_ravia_similarity_mols_layers,df_ravia_similarity_mols_layers_zscored,df_ravia_similarity_mols_embeddings_linear\
# ,df_ravia_similarity_mols_embeddings_linear_zscored=prepare_ravia_similarity_mols2(df_ravia_similarity_mean,lm,tokenizer,modeldeepchem_gslf=None)

In [ ]:
def cosine_sim_helper(df_mols_embeddings,df_mols_embeddings_zscored,df_mols_layers,df_mols_layers_zscored,df_mols_embeddings_linear,df_mols_embeddings_linear_zscored):
    
    cosine_sim_df_mols_embeddings=cosine_similarity_df(df_mols_embeddings,'Combined')
    cosine_sim_df_mols_embeddings_zscored=cosine_similarity_df(df_mols_embeddings_zscored,'Combined')
    
    
    cosine_sim_df_mols_embeddings_linear=cosine_similarity_df(df_mols_embeddings_linear,'Combined')
    cosine_sim_df_mols_embeddings_linear_zscored=cosine_similarity_df(df_mols_embeddings_linear_zscored,'Combined')
    
    ### Cosine similarity for all layers
    cosine_sim_df_mols_layers = []
    cosine_sim_df_mols_layers_zscored = []
    # embedding = molecules_activations_embeddings_original[0]
    for embeddings in df_mols_layers:
        cosine_sim_df_mols_layers.append(cosine_similarity_df(embeddings,'Combined'))
        
    for embeddings in df_mols_layers_zscored:
        cosine_sim_df_mols_layers_zscored.append(cosine_similarity_df(embeddings,'Combined'))
        
    return cosine_sim_df_mols_embeddings, cosine_sim_df_mols_embeddings_zscored, cosine_